## 3️⃣ Natural Language to SQL Queries

### 1 Creating the Chinook database

In [45]:
import sqlite3

db_path = 'Chinook.db'
chinook_sql_path = 'data/Chinook_Sqlite.sql'  

with open(chinook_sql_path, 'r', encoding='utf-8') as f:
    sql_script = f.read()
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.executescript(sql_script)
    conn.commit()
print('Chinook database schema and data loaded using sqlite3.')
# Check if data is present in the Artist table
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT COUNT(*) FROM Artist')
    print('Number of rows in Artist table:', cursor.fetchone()[0])

Chinook database schema and data loaded using sqlite3.
Number of rows in Artist table: 275


### 2 loading the Chinook database into SQLite and Explore the Schema

In [37]:


# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('Chinook.db')
cursor = conn.cursor()

# Exract and display the schema of the database
def display_schema(conn):
    
    schema = ""

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables  = [row[0] for row in cursor.fetchall() if row[0] != 'sqlite_sequence']

    for table in tables:
        schema += f"table: {table}\nColumns:\n"
        cursor.execute(f"PRAGMA table_info({table})")

        for column in cursor.fetchall():
            schema += f"  - {column[1]} ({column[2]})\n"
        schema += "\n"
    return schema
schema = display_schema(cursor)
print(schema[:1000])  # Display the first 1000 characters of the schema


table: Album
Columns:
  - AlbumId (INTEGER)
  - Title (NVARCHAR(160))
  - ArtistId (INTEGER)

table: Artist
Columns:
  - ArtistId (INTEGER)
  - Name (NVARCHAR(120))

table: Customer
Columns:
  - CustomerId (INTEGER)
  - FirstName (NVARCHAR(40))
  - LastName (NVARCHAR(20))
  - Company (NVARCHAR(80))
  - Address (NVARCHAR(70))
  - City (NVARCHAR(40))
  - State (NVARCHAR(40))
  - Country (NVARCHAR(40))
  - PostalCode (NVARCHAR(10))
  - Phone (NVARCHAR(24))
  - Fax (NVARCHAR(24))
  - Email (NVARCHAR(60))
  - SupportRepId (INTEGER)

table: Employee
Columns:
  - EmployeeId (INTEGER)
  - LastName (NVARCHAR(20))
  - FirstName (NVARCHAR(20))
  - Title (NVARCHAR(30))
  - ReportsTo (INTEGER)
  - BirthDate (DATETIME)
  - HireDate (DATETIME)
  - Address (NVARCHAR(70))
  - City (NVARCHAR(40))
  - State (NVARCHAR(40))
  - Country (NVARCHAR(40))
  - PostalCode (NVARCHAR(10))
  - Phone (NVARCHAR(24))
  - Fax (NVARCHAR(24))
  - Email (NVARCHAR(60))

table: Genre
Columns:
  - GenreId (INTEGER)
  - Name (

### 3 creating a prompt template for the LLM

In [38]:
import google.generativeai as genai
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Set the API key for Google Generative AI

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE-API-KEY")

prompt = PromptTemplate.from_template(
    """ 
Your are a SQL assistance. Given a database chema and a natural language schema,
write the correct SQL query to answer it. Only use the given schema.
Schema:
{schema}
Question: "{question}"

SQL:
"""
)


### 4 Prompting the AI agent

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable

# Use the Gemini 2.0 Flash model for the LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
output_parser = StrOutputParser()

# Chain: prompt -> LLM -> output parser
chain: Runnable = prompt | llm | output_parser

### 5 Parsing the LLM SQL Output and Execute it

In [40]:
def process_question(question: str):
    sql_query = chain.invoke({"schema": schema, "question": question}).strip()
    # Remove markdown code block markers if present
    if sql_query.startswith('```sql') and sql_query.endswith('```'):
        sql_query = sql_query[6:-3].strip()
    elif sql_query.startswith('```') and sql_query.endswith('```'):
        sql_query = sql_query[3:-3].strip()
    try:
        result = pd.read_sql_query(sql_query, conn)
    except Exception as e:
        result = f"Error: {e}"
    return sql_query, result

### 6 Testing with few questions

In [48]:
import pandas as pd
from IPython.display import display
# Set pandas display options for better notebook output
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

questions = [
    "List all customers in Canada.",
    "Which employees are sales agents?",
    "What are the 5 most purchased tracks?",
    "Show total sales per country in decending order.",
    "Which artists have more than 5 albums?"
    "list three expensive albums",
    "What is the total revenue from sales?",
 ]
for question in questions:
    sql_query, result = process_question(question)
    print(f"\nQuestion: {question}")
    print(f"SQL Query: {sql_query}")
    print("Result:")
    if isinstance(result, pd.DataFrame):
        display(result)  # Show as scrollable table in notebook
    else:
        print(result)
    print("\n" + "="*100 + "\n")


Question: List all customers in Canada.
SQL Query: SELECT
  CustomerId,
  FirstName,
  LastName
FROM Customer
WHERE
  Country = 'Canada';
Result:


,CustomerId,FirstName,LastName
0,3,François,Tremblay
1,14,Mark,Philips
2,15,Jennifer,Peterson
3,29,Robert,Brown
4,30,Edward,Francis
5,31,Martha,Silk
6,32,Aaron,Mitchell
7,33,Ellie,Sullivan





Question: Which employees are sales agents?
SQL Query: SELECT
  EmployeeId,
  FirstName,
  LastName
FROM Employee
WHERE
  Title = 'Sales Support Agent';
Result:

Question: Which employees are sales agents?
SQL Query: SELECT
  EmployeeId,
  FirstName,
  LastName
FROM Employee
WHERE
  Title = 'Sales Support Agent';
Result:


,EmployeeId,FirstName,LastName
0,3,Jane,Peacock
1,4,Margaret,Park
2,5,Steve,Johnson





Question: What are the 5 most purchased tracks?
SQL Query: SELECT
  T.Name
FROM Track AS T
JOIN InvoiceLine AS IL
  ON T.TrackId = IL.TrackId
GROUP BY
  T.Name
ORDER BY
  SUM(IL.Quantity) DESC
LIMIT 5;
Result:

Question: What are the 5 most purchased tracks?
SQL Query: SELECT
  T.Name
FROM Track AS T
JOIN InvoiceLine AS IL
  ON T.TrackId = IL.TrackId
GROUP BY
  T.Name
ORDER BY
  SUM(IL.Quantity) DESC
LIMIT 5;
Result:


,Name
0,The Trooper
1,Untitled
2,The Number Of The Beast
3,Sure Know Something
4,Hallowed Be Thy Name





Question: Show total sales per country in decending order.
SQL Query: SELECT
  BillingCountry,
  SUM(Total)
FROM Invoice
GROUP BY
  BillingCountry
ORDER BY
  SUM(Total) DESC;
Result:

Question: Show total sales per country in decending order.
SQL Query: SELECT
  BillingCountry,
  SUM(Total)
FROM Invoice
GROUP BY
  BillingCountry
ORDER BY
  SUM(Total) DESC;
Result:


,BillingCountry,SUM(Total)
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
...,...,...
19,Italy,37.62
20,Denmark,37.62
21,Belgium,37.62
22,Australia,37.62





Question: Which artists have more than 5 albums?list three expensive albums
SQL Query: SELECT
  T1.Name
FROM Artist AS T1
INNER JOIN Album AS T2
  ON T1.ArtistId = T2.ArtistId
GROUP BY
  T1.Name
HAVING
  COUNT(T2.AlbumId) > 5;
SELECT
  Title
FROM Album
ORDER BY
  AlbumId DESC
LIMIT 3;
Result:
Error: Execution failed on sql 'SELECT
  T1.Name
FROM Artist AS T1
INNER JOIN Album AS T2
  ON T1.ArtistId = T2.ArtistId
GROUP BY
  T1.Name
HAVING
  COUNT(T2.AlbumId) > 5;
SELECT
  Title
FROM Album
ORDER BY
  AlbumId DESC
LIMIT 3;': You can only execute one statement at a time.



Question: Which artists have more than 5 albums?list three expensive albums
SQL Query: SELECT
  T1.Name
FROM Artist AS T1
INNER JOIN Album AS T2
  ON T1.ArtistId = T2.ArtistId
GROUP BY
  T1.Name
HAVING
  COUNT(T2.AlbumId) > 5;
SELECT
  Title
FROM Album
ORDER BY
  AlbumId DESC
LIMIT 3;
Result:
Error: Execution failed on sql 'SELECT
  T1.Name
FROM Artist AS T1
INNER JOIN Album AS T2
  ON T1.ArtistId = T2.ArtistId
GROUP 

,sum(Total)
0,2328.6
